In [1]:
pip install torch==2.0.1 torchtext==0.15.2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [4]:
!pip install streamlit pandas torch torchtext pyngrok --quiet


# **`Transformer From Scratch`**

In [ ]:
import torch
import torch.nn as nn
import math

class LayerNormalization(nn.Module):

    def __init__(self, features: int, eps:float=10**-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features)) # alpha is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features)) # bias is a learnable parameter

    def forward(self, x):
        # x: (batch, seq_len, hidden_size)
         # Keep the dimension for broadcasting
        mean = x.mean(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # Keep the dimension for broadcasting
        std = x.std(dim = -1, keepdim = True) # (batch, seq_len, 1)
        # eps is to prevent dividing by zero or when std is very small
        return self.alpha * (x - mean) / (std + self.eps) + self.bias

class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

    def forward(self, x):
        # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

class InputEmbeddings(nn.Module):

    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        # (batch, seq_len) --> (batch, seq_len, d_model)
        # Multiply by sqrt(d_model) to scale the embeddings according to the paper
        return self.embedding(x) * math.sqrt(self.d_model)

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
        # Create a vector of shape (d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)

class ResidualConnection(nn.Module):

        def __init__(self, features: int, dropout: float) -> None:
            super().__init__()
            self.dropout = nn.Dropout(dropout)
            self.norm = LayerNormalization(features)

        def forward(self, x, sublayer):
            return x + self.dropout(sublayer(self.norm(x)))

class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads
        # Make sure d_model is divisible by h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
        self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
        self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]
        # Just apply the formula from the paper
        # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            # Write a very low value (indicating -inf) to the positions where mask == 0
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
        # return attention scores which can be used for visualization
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        key = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)

        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

        # Combine all the heads together
        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # Multiply by Wo
        # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        return self.w_o(x)

class EncoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

class Encoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class DecoderBlock(nn.Module):

    def __init__(self, features: int, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(features, dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

class Decoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

class ProjectionLayer(nn.Module):

    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
        return self.proj(x)

class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # (batch, seq_len, d_model)
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        # (batch, seq_len, d_model)
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def project(self, x):
        # (batch, seq_len, vocab_size)
        return self.projection_layer(x)

def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
    # Create the embedding layers
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Create the positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Create the decoder blocks
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    # Create the encoder and decoder
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))

    # Create the projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    # Create the transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

# **Model Training**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torchtext.vocab import build_vocab_from_iterator


# 1. DATA PREPROCESSING


# Load dataset
df_train = pd.read_csv("/content/spoc-train.csv")
df_train.dropna(inplace=True)

# Convert to list of tuples (pseudocode, code)
data_pairs = list(zip(df_train[' text'].tolist(), df_train['code'].tolist()))

# Tokenization function (character-wise, preserving spaces)
def tokenize_text(text):
    return list(text)

# Build vocabularies
special_tokens = ["<pad>", "<sos>", "<eos>"]
src_vocab = build_vocab_from_iterator((tokenize_text(pair[0]) for pair in data_pairs), specials=special_tokens)
tgt_vocab = build_vocab_from_iterator((tokenize_text(pair[1]) for pair in data_pairs), specials=special_tokens)

src_vocab.set_default_index(src_vocab["<pad>"])
tgt_vocab.set_default_index(tgt_vocab["<pad>"])

# 2. CUSTOM DATASET CLASS


class CodeDataset(Dataset):
    def __init__(self, data_pairs, src_vocab, tgt_vocab, src_max_len=200, tgt_max_len=200):
        self.data_pairs = data_pairs
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab
        self.src_max_len = src_max_len
        self.tgt_max_len = tgt_max_len

    def __len__(self):
        return len(self.data_pairs)

    def __getitem__(self, idx):
        src_text, tgt_text = self.data_pairs[idx]

        # Tokenize and convert to indices
        src_tokens = ["<sos>"] + tokenize_text(src_text) + ["<eos>"]
        tgt_tokens = ["<sos>"] + tokenize_text(tgt_text) + ["<eos>"]

        src_indices = [self.src_vocab[token] for token in src_tokens][:self.src_max_len]
        tgt_indices = [self.tgt_vocab[token] for token in tgt_tokens][:self.tgt_max_len]

        # Pad sequences
        src_indices += [self.src_vocab["<pad>"]] * (self.src_max_len - len(src_indices))
        tgt_indices += [self.tgt_vocab["<pad>"]] * ((self.tgt_max_len + 1) - len(tgt_indices))

        return torch.tensor(src_indices), torch.tensor(tgt_indices)

# Create dataset and dataloader
dataset = CodeDataset(data_pairs, src_vocab, tgt_vocab)
subset_size = 30000
batch_size = 32

train_subset = torch.utils.data.Subset(dataset, range(subset_size))
dataloader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)

if len(dataloader) == 0:
    print("🚨 ERROR: DataLoader is empty! Check dataset loading.")
    exit()

print(f"✅ DataLoader initialized with {len(dataloader)} batches.")


# 3. MODEL INITIALIZATION


class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, N, h, dropout, d_ff):
        super().__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)

        self.positional_encoding = nn.Parameter(torch.zeros(1, 200, d_model))  # Fixed position embeddings

        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=h,
            num_encoder_layers=N,
            num_decoder_layers=N,
            dim_feedforward=d_ff,
            dropout=dropout,
            batch_first=True
        )

        self.fc_out = nn.Linear(d_model, tgt_vocab_size)

    def generate_square_subsequent_mask(self, size):
        """ Creates a lower-triangular mask for the decoder """
        mask = torch.triu(torch.ones(size, size), diagonal=1)
        return mask.masked_fill(mask == 1, float('-inf'))

    def forward(self, src, tgt):
        src_mask = None
        tgt_mask = self.generate_square_subsequent_mask(tgt.shape[1]).to(src.device)

        src_emb = self.encoder_embedding(src) + self.positional_encoding[:, :src.shape[1], :]
        tgt_emb = self.decoder_embedding(tgt) + self.positional_encoding[:, :tgt.shape[1], :]

        output = self.transformer(src_emb, tgt_emb, src_mask=src_mask, tgt_mask=tgt_mask)
        return self.fc_out(output)

# Model hyperparameters
d_model = 256
h = 8
N = 4
d_ff = 512
dropout = 0.1
src_vocab_size = len(src_vocab)
tgt_vocab_size = len(tgt_vocab)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {device}")

model = TransformerModel(src_vocab_size, tgt_vocab_size, d_model, N, h, dropout, d_ff).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=src_vocab["<pad>"])
optimizer = optim.Adam(model.parameters(), lr=3e-4)


# 4. TRAINING FUNCTION


def train(model, dataloader, criterion, optimizer, num_epochs=10):
    model.train()
    print("🚀 Training Started...")

    for epoch in range(num_epochs):
        total_loss = 0
        print(f"\n🔵 Epoch {epoch+1}/{num_epochs}")

        for i, (src, tgt) in enumerate(dataloader):
            src, tgt = src.to(device), tgt.to(device)
            optimizer.zero_grad()
            print(f"🟡 Batch {i+1}/{len(dataloader)} - Src Shape: {src.shape}, Tgt Shape: {tgt.shape}")
            # Shift tgt by one position to the right
            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:].contiguous().view(-1)

            output = model(src, tgt_input)
            output = output.view(-1, tgt_vocab_size)

            loss = criterion(output, tgt_output)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            total_loss += loss.item()

        print(f"✅ Epoch [{epoch+1}/{num_epochs}], Avg Loss: {total_loss/len(dataloader):.4f}")

    print("🎉 Training Complete!")

# Train the model
train(model, dataloader, criterion, optimizer, num_epochs=5)

# Save the model
torch.save(model.state_dict(), "transformer_model.pth")
print("✅ Model saved successfully!")


✅ DataLoader initialized with 938 batches.
✅ Using device: cpu
🚀 Training Started...

🔵 Epoch 1/5
🟡 Batch 1/938 - Src Shape: torch.Size([32, 200]), Tgt Shape: torch.Size([32, 201])
🟡 Batch 2/938 - Src Shape: torch.Size([32, 200]), Tgt Shape: torch.Size([32, 201])
🟡 Batch 3/938 - Src Shape: torch.Size([32, 200]), Tgt Shape: torch.Size([32, 201])
🟡 Batch 4/938 - Src Shape: torch.Size([32, 200]), Tgt Shape: torch.Size([32, 201])
🟡 Batch 5/938 - Src Shape: torch.Size([32, 200]), Tgt Shape: torch.Size([32, 201])
🟡 Batch 6/938 - Src Shape: torch.Size([32, 200]), Tgt Shape: torch.Size([32, 201])
🟡 Batch 7/938 - Src Shape: torch.Size([32, 200]), Tgt Shape: torch.Size([32, 201])
🟡 Batch 8/938 - Src Shape: torch.Size([32, 200]), Tgt Shape: torch.Size([32, 201])
🟡 Batch 9/938 - Src Shape: torch.Size([32, 200]), Tgt Shape: torch.Size([32, 201])
🟡 Batch 10/938 - Src Shape: torch.Size([32, 200]), Tgt Shape: torch.Size([32, 201])
🟡 Batch 11/938 - Src Shape: torch.Size([32, 200]), Tgt Shape: torch.Siz

# **Streamlit code for creating App.py**

In [47]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
from torchtext.vocab import build_vocab_from_iterator
import pandas as pd
import base64
from PIL import Image

# 1. COVER PAGE
image_path = "/content/PseudotoCppCover.jpg"
def get_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode()
base64_image = get_base64(image_path)
st.set_page_config(page_title="Pseudocode to C++ Converter", layout="centered")
st.markdown(
    f"""
    <style>
    [data-testid="stAppViewContainer"] {{
        background-image: url("data:image/jpeg;base64,{base64_image}");
        background-size: cover;
        background-position: center;
        background-repeat: no-repeat;
    }}
    [data-testid="stAppViewContainer"] {{
        color: rgb(0, 0, 0);  !important;
    }}
    </style>
    """,
    unsafe_allow_html=True
)


st.title("Pseudocode to C++ Code Generator")
st.write("Convert pseudocode into C++ code using a Transformer model.")


# 2. LOAD MODEL & VOCAB


def tokenize_text(text):
    return list(text)

# Load dataset (for vocab)
df_train = pd.read_csv("spoc-train.csv")
df_train.dropna(inplace=True)
data_pairs = list(zip(df_train[' text'].tolist(), df_train['code'].tolist()))

# Build vocabularies
special_tokens = ["<pad>", "<sos>", "<eos>"]
src_vocab = build_vocab_from_iterator((tokenize_text(pair[0]) for pair in data_pairs), specials=special_tokens)
tgt_vocab = build_vocab_from_iterator((tokenize_text(pair[1]) for pair in data_pairs), specials=special_tokens)

src_vocab.set_default_index(src_vocab["<pad>"])
tgt_vocab.set_default_index(tgt_vocab["<pad>"])
idx_to_tgt = {idx: token for token, idx in tgt_vocab.get_stoi().items()}

class TransformerModel(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, N, h, dropout, d_ff):
        super().__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = nn.Parameter(torch.zeros(1, 200, d_model))
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=h,
            num_encoder_layers=N,
            num_decoder_layers=N,
            dim_feedforward=d_ff,
            dropout=dropout,
            batch_first=True
        )
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)

    def generate_square_subsequent_mask(self, size):
        mask = torch.triu(torch.ones(size, size), diagonal=1)
        return mask.masked_fill(mask == 1, float('-inf'))

    def forward(self, src, tgt):
        src_mask = None
        tgt_mask = self.generate_square_subsequent_mask(tgt.shape[1]).to(src.device)
        src_emb = self.encoder_embedding(src) + self.positional_encoding[:, :src.shape[1], :]
        tgt_emb = self.decoder_embedding(tgt) + self.positional_encoding[:, :tgt.shape[1], :]
        output = self.transformer(src_emb, tgt_emb, src_mask=src_mask, tgt_mask=tgt_mask)
        return self.fc_out(output)

# Load model
model = TransformerModel(len(src_vocab), len(tgt_vocab), 256, 4, 8, 0.1, 512)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("transformer_model.pth", map_location=device))
model.to(device)
model.eval()


# 3. TRANSLATION FUNCTION

def translate_pseudocode(pseudocode, max_length=200):
    model.eval()
    src_tokens = ["<sos>"] + tokenize_text(pseudocode) + ["<eos>"]
    src_indices = [src_vocab[token] for token in src_tokens] + [src_vocab["<pad>"]] * (max_length - len(src_tokens))
    src_tensor = torch.tensor([src_indices], device=device)
    tgt_indices = [tgt_vocab["<sos>"]]
    tgt_tensor = torch.tensor([tgt_indices], device=device)

    for _ in range(max_length):
        with torch.no_grad():
            output = model(src_tensor, tgt_tensor)
            next_token = output[:, -1, :].argmax(dim=-1).item()
        tgt_indices.append(next_token)
        if next_token == tgt_vocab["<eos>"]:
            break
        tgt_tensor = torch.tensor([tgt_indices], device=device)

    return "".join(idx_to_tgt[idx] for idx in tgt_indices[1:-1])


# 4. STREAMLIT UI


st.subheader("Convert Pseudocode to C++")
pseudocode_input = st.text_area("Enter Pseudocode:", "in the function gcd(a,b=integers)")
if st.button("Convert"):
    with st.spinner("Generating C++ Code..."):
        generated_code = translate_pseudocode(pseudocode_input)
    st.success("Generated C++ Code:")
    st.code(generated_code, language="cpp")


Overwriting app.py


# **Run and  copy the ouput address and then Paste in the url generated**

In [48]:
!wget -q -O - ipv4.icanhazip.com

34.169.32.13


# **Run Streamlit App**

In [49]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.169.32.13:8501

⠼⠴your url is: https://hip-poets-shout.loca.lt
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/activation.py:1160: User